In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [16]:
csv1 = "../movie_data/movies.csv"
csv2 = "../movie_data/ratings.csv"
csv3 = "../movie_data/genome-scores.csv"
csv4 = "../movie_data/genome-tags.csv"
# csv5 = "movie_data/links.csv"
# csv6 = "movie_data/tags.csv"

In [5]:
# movies_df = pd.read_csv(csv1)
ratings_df = pd.read_csv(csv2)
genome_scores_df = pd.read_csv(csv3)


In [6]:
print(len(ratings_df))

25000095


In [7]:
print(len(genome_scores_df))

15584448


In [8]:
reduced_ratings_df = ratings_df.loc[(ratings_df['movieId'] < 10)]

In [9]:
reduced_genome_scores_df = genome_scores_df.loc[(genome_scores_df['movieId'] < 10)]

In [10]:
print(len(reduced_ratings_df))

149353


In [11]:
print(len(reduced_genome_scores_df))

10152


In [120]:
relevant_genomes = reduced_genome_scores_df.loc[reduced_genome_scores_df['relevance'] > .85]

In [121]:
new_df = reduced_ratings_df.merge(relevant_genomes, how="inner")
new_df.head()

,userId,movieId,rating,timestamp,tagId,relevance
0,2,1,3.5,1141415820,29,0.89375
1,2,1,3.5,1141415820,63,0.94725
2,2,1,3.5,1141415820,64,0.98425
3,2,1,3.5,1141415820,186,0.95475
4,2,1,3.5,1141415820,203,0.90375


In [122]:
new_df["rating"] = np.where(new_df["rating"] >= 3, 1, 0)

In [123]:
new_df.head()

,userId,movieId,rating,timestamp,tagId,relevance
0,2,1,1,1141415820,29,0.89375
1,2,1,1,1141415820,63,0.94725
2,2,1,1,1141415820,64,0.98425
3,2,1,1,1141415820,186,0.95475
4,2,1,1,1141415820,203,0.90375


In [14]:
print(len(new_df))

2194891


In [30]:
new_df.to_csv("movie_data/reduced.csv")

In [77]:
movies_df = pd.read_csv(csv1)
tags_df = pd.read_csv(csv4)

In [78]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [79]:
movies_df['year'] = movies_df.title.str.extract('\((.*)\)')[0]
#movies_df = df[df['Location'].notnull()]
#df['Injury/Rest'] = np.where(df['Location'].eq('rest'), 'Rest', 'Injury')

In [85]:
movies_df['release_year'] = movies_df['year'].str.split('(').str[-1]

In [91]:
movies_df = movies_df.drop(columns = ['year'])

In [95]:
movies_df.head()

,movieId,title,genres,release_year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [124]:
final_df = new_df.merge(movies_df, how="inner")

In [125]:
final_df = final_df.drop(columns = ['title','genres'])

In [126]:
final_df

,userId,movieId,rating,timestamp,tagId,relevance,release_year
0,2,1,1,1141415820,29,0.89375,1995
1,2,1,1,1141415820,63,0.94725,1995
2,2,1,1,1141415820,64,0.98425,1995
3,2,1,1,1141415820,186,0.95475,1995
4,2,1,1,1141415820,203,0.90375,1995
...,...,...,...,...,...,...,...
2194886,162266,8,1,845976277,22,0.92800,1995
2194887,162266,8,1,845976277,107,0.92950,1995
2194888,162491,8,1,833659748,21,0.85450,1995
2194889,162491,8,1,833659748,22,0.92800,1995


In [127]:
X = final_df.drop('rating', axis=1)
y = final_df['rating']

In [128]:
from sklearn import preprocessing
from sklearn import utils

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

#view transformed values
print(y_transformed)
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, random_state=1)

[1 1 1 ... 1 1 1]


In [129]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.891271121781009
Testing Data Score: 0.8912365619811817
